In [1]:
! pip install tiktoken
! pip install matplotlib
! pip install wordcloud
! pip install pandas
! pip install nltk

  Using cached urllib3-2.5.0-py3-none-any.whl.metadata (6.5 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 994.0/994.0 kB 31.3 MB/s eta 0:00:00
Using cached urllib3-2.5.0-py3-none-any.whl (129 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5/5 [tiktoken]

[notice] A new release of pip is available: 25.1.1 -> 25.3
[notice] To update, run: pip install --upgrade pip
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.1/8.1 MB 34.2 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 49.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.1/5.1 MB 45.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.7/4.7 MB 43.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8/8 [matplotlib]8 [matplotlib]

[notice] A new release of pip is available: 25.1.1 -> 25.3
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 25.1.1 -> 25.3
[notice] To update, run: pip install --upgrade pip
  Using cached pyt

In [ ]:
import pandas as pd

df = pd.read_csv('../prompts.csv')


,act,prompt,for_devs
count,220,220,220
unique,216,220,2
top,Chess Player,Imagine you are an experienced Ethereum develo...,False
freq,2,1,167


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 220 entries, 0 to 219
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   act       220 non-null    object
 1   prompt    220 non-null    object
 2   for_devs  220 non-null    bool  
dtypes: bool(1), object(2)
memory usage: 3.8+ KB


In [7]:
act_count = df['act'].value_counts()
for_dev_count = df['for_devs'].value_counts()
print(act_count)
print(for_dev_count)

act
Chess Player                2
Life Coach                  2
Python Interpreter          2
Prompt Generator            2
Ethereum Developer          1
                           ..
Journal Reviewer            1
DIY Expert                  1
Social Media Influencer     1
Socrat                      1
Explainer with Analogies    1
Name: count, Length: 216, dtype: int64
for_devs
False    167
True      53
Name: count, dtype: int64


In [18]:
len(df["act"])


220

In [19]:
len(df["act"].unique())

216

In [20]:
df.drop_duplicates(inplace=True)

In [21]:
null_values = df.isnull().sum()
print(null_values)

act         0
prompt      0
for_devs    0
dtype: int64
